# Exercise: Business cycle correlations

For this exercise, you'll be using macroeconomic data from the folder `../data/FRED`.

1.  There are seven decade-specific files named `FRED_monthly_19X0.csv` where `X` identifies the decade (`X` takes on the values 5, 6, 7, 8, 9, 0, 1). Write a loop that reads in all seven files as DataFrames and store them in a list.

    *Hint:* Recall from the lecture that you should use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
2.  Use [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) to concate these data sets into a single `DataFrame` and set the `DATE` column as the index.
3.  You realize that your data does not include GDP since this variable is only reported at quarterly frequency.
    Load the GDP data from the file `GDP.csv` and merge it with your monthly data using an _inner join_.
4.  You want to compute how (percent) changes of the variables in your data correlate with percent changes in GDP.

    1. Create a _new_ `DataFrame` which contains the percent changes in CPI and GDP (using 
    [`pct_change()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html), 
    see also the last exercise in workshop 3),
    and the absolute changes for the remaining variables (using 
    [`diff()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html)).
    2.  Compute the correlation of the percent changes in GDP with the (percent) changes of all other variables (using [`corr()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)). What does the sign and magnitude of the correlation coefficient tell you?



In [210]:
import pandas as pd 
import numpy as np
import os.path

DATA_PATH = '/Users/tobiaswarland/NHH/Tech2-local/TECH2-H24/data/FRED/'

data  = []

years = np.arange(1950, 2011, 10)

for year in years:
    fn = f'FRED_monthly_{year}.csv'

    path = os.path.join(DATA_PATH, fn)

    df = pd.read_csv(path, parse_dates=['DATE'])

    data.append(df)

df = pd.concat(data, axis= 0).reset_index(drop= True).set_index('DATE')

df

,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART
DATE,,,,,
1950-01-01,23.5,6.5,NaN,NaN,58.9
1950-02-01,23.6,6.4,NaN,NaN,58.9
1950-03-01,23.6,6.3,NaN,NaN,58.8
1950-04-01,23.6,5.8,NaN,NaN,59.2
1950-05-01,23.8,5.5,NaN,NaN,59.1
...,...,...,...,...,...
2019-08-01,256.0,3.6,2.1,0.6,63.1
2019-09-01,256.4,3.5,2.0,0.3,63.2
2019-10-01,257.2,3.6,1.8,-0.0,63.3


## Part 3 

In [219]:

fn = os.path.join(DATA_PATH, 'GDP.csv')

gdp = pd.read_csv(fn, parse_dates=['DATE'], index_col='DATE')

df = df.join(gdp, on= 'DATE', how = 'inner')


In [235]:
newdf = df[['CPI', 'GDP']].pct_change()*100

abschanges = df[['UNRATE', 'FEDFUNDS', 'REALRATE', 'LFPART']].diff()

abschanges

dfcorrelations = df.corr()

dfcorrelations['GDP']


CPI         0.991718
UNRATE      0.133377
FEDFUNDS   -0.388324
REALRATE   -0.798474
LFPART      0.683415
GDP         1.000000
Name: GDP, dtype: float64

***
# Exercise: Loading many data files

In the previous exercise, you loaded the individual files by specifing an explicit list of file names. This can become tedious or infeasible if your data is spread across many files with varying file name patterns. Python offers the possibility to iterate over all files in a directory (for example, using [`os.listdir()`](https://docs.python.org/3/library/os.html#os.listdir)),
or to iterate over files that match a pattern, for example using [`glob.glob()`](https://docs.python.org/3/library/glob.html).

Repeat parts (1) and (2) from the previous exercise, but now iterate over the input files using 
[`glob.glob()`](https://docs.python.org/3/library/glob.html). You'll need to use a wildcard `*` and make sure to match only the relevant files in `../data/FRED`, i.e., those that start with `FRED_monthly`.

In [16]:
from datetime import datetime
import pandas as pd 
import numpy as np
import os.path
import glob

DATA_PATH = '/Users/tobiaswarland/NHH/Tech2-local/TECH2-H24/data/FRED/'
pattern = f"{DATA_PATH}/FRED_monthly*"


years  = glob.glob(pattern)

data = []
for year in years:
    df = pd.read_csv(year, parse_dates=['DATE'])
    
    data.append(df)

df = pd.concat(data, axis= 0).reset_index(drop= True).set_index('DATE')

df.sort_index(inplace=True)



,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART
DATE,,,,,
1950-01-01,23.5,6.5,NaN,NaN,58.9
1950-02-01,23.6,6.4,NaN,NaN,58.9
1950-03-01,23.6,6.3,NaN,NaN,58.8
1950-04-01,23.6,5.8,NaN,NaN,59.2
1950-05-01,23.8,5.5,NaN,NaN,59.1
...,...,...,...,...,...
2019-08-01,256.0,3.6,2.1,0.6,63.1
2019-09-01,256.4,3.5,2.0,0.3,63.2
2019-10-01,257.2,3.6,1.8,-0.0,63.3


***
# Exercise: Decade averages of macro time series


For this exercise, you'll be using macroeconomic data from the folder `../data/FRED`.

1.  There are five files containing monthly observations on annual inflation (INFLATION), the Fed Funds rate (FEDFUNDS), the labor force participation rate (LFPART), the 1-year real interest rate (REALRATE) and the unemployment rate (UNRATE).
    Write a loop to import these and merge them on `DATE` into a single `DataFrame` using _outer joins_ (recall that [`merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) 
    and [`join()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html) operate on only two DataFrames at a time). 

    *Hint:* Recall from the lecture that you should use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.

2.  Your friend is a pandas guru and tells you that you don't need to iteratively merge many files but can instead directly use [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) for merging many DataFrames in a single step.
    Repeat the previous part using `pd.concat()` instead, and verify that you get the same result (you can do this using [`compare()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.compare.html)).
3.  You want to compute the average value of each variable by decade, but you want to include only decades without _any_ missing values for _all_ variables.
    1.  Create a variable `Decade` which stores the decade (1940, 1950, ...) for each observation.

        *Hint:* You should have set the `DATE` as the `DataFrame` index. Then you can access the calendar year using the attribute `df.index.year` which can be used to compute the decade.
    2.  Write a function `num_missing(x)` which takes as argument `x` a `Series` and returns the number of missing values in this `Series`.
    3.  Compute the number of missing values by decade for each variable using a [`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) operation and the function `num_missing` you wrote.
    4.  Aggregate this data across all variables to create an indicator for each decade whether there are any missing values. This can be done in many ways but will require aggregation across columns, e.g., with `sum(..., axis=1)`.
    5.  Merge this decade-level indicator data back into the original `DataFrame` (_many-to-one_ merge). 
4.  Using this indicator, drop all observations which are in a decade with missing values.
5.  Compute the decade average for each variable.

**Challenge**

-   Your pandas guru friend claims that all the steps in 3.2 to 3.5 can be done with a single one-liner using [`transform()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.transform.html). Can you come up with a solution?

    

In [41]:
import pandas as pd 
import numpy as nd 
import os.path
import glob 

DATA_PATH = '/Users/tobiaswarland/NHH/Tech2-local/TECH2-H24/data/FRED'

pattern = f"{DATA_PATH}/*"

pathlist = glob.glob(pattern)

data = []

for path in pathlist:
    if path != f'{DATA_PATH}/FRED':
        df = pd.read_csv(path, parse_dates = ['DATE'])
        df.merge(df,how= 'outer').reset_index()


    

df

,DATE,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART
0,1960-01-01,29.4,5.2,4.0,NaN,59.1
1,1960-02-01,29.4,4.8,4.0,NaN,59.1
2,1960-03-01,29.4,5.4,3.8,NaN,58.5
3,1960-04-01,29.5,5.2,3.9,NaN,59.5
4,1960-05-01,29.6,5.1,3.8,NaN,59.5
...,...,...,...,...,...,...
115,1969-08-01,36.9,3.5,9.2,NaN,60.3
116,1969-09-01,37.1,3.7,9.2,NaN,60.3
117,1969-10-01,37.3,3.7,9.0,NaN,60.4
118,1969-11-01,37.5,3.5,8.8,NaN,60.2


***
# Exercise: Mering the Titanic data

In this exercise, you'll be working with the the original Titanic data set in `titanic.csv` and additional (partly fictitious) information on passengers stored in `titanic-additional.csv`, both located in the `data/` folder.

The goal of the exercise is to calculate the survival rates by country of residence (for this exercise we restrict ourselves to the UK, so these will be England, Scotland, etc.).

1.  Load the `titanic.csv` and `titanic-additional.csv` into two DataFrames.

    Inspect the columns contained in both data sets. As you can see, the original data contains the full name including the title
    and potentially maiden name (for married women) in a single column.
    The additional data contains this information in separate columns.
    You want to merge these data sets, but you first need to create common keys in both DataFrames.

2.  Since the only common information is the name, you'll need to extract the individual name components from the original DataFrame
    and use these as merge keys.

    Focusing only on men (who have names that are much easier to parse), split the `Name` column into the tokens 
    `Title`, `FirstName` and `LastName`, just like the columns in the second DataFrame.

    *Hint:* This is the same task as in the last exercise in Workshop 2. You can just use your solution here.

3.  Merge the two data sets based on the columns `Title`, `FirstName` and `LastName` you just created using a _left join_ (_one-to-one_ merge).
    Tabulate the columns and the number of non-missing observations to make sure that merging worked. 

    *Note:* The additional data set contains address information only for passengers from the UK, so some of these fields will be missing.

4.  You are now in a position to merge the country of residence (_many-to-one_ merge). Load the country data from `UK_post_codes.csv` which contains 
    the UK post code prefix (which you can ignore), the corresponding city, and the corresponding country.

    Merge this data with your passenger data set using a _left join_ (what is the correct merge key?).

5.  Tabulate the number of observations by `Country`, including the number of observations with missing `Country` (these are passengers residing outside the UK).

    Finally, compute the mean survival rate by country.